In [1]:
import arviz as az
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as scs
import itertools as iter
from IPython.display import display
from pytensor.printing import pydotprint


%config InlineBackend.figure_format = 'retina'
# Initialize random number generator
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

%load_ext autoreload
%autoreload 2
import whr

%load_ext pyinstrument

In [2]:
print('Elo = natural rating * %.1f + 1500' % whr.eloPerNaturalRating)

Elo = natural rating * 173.7 + 1500


In [3]:
da = whr.PreprocessedData.load()

# print('-1 in first row for each player in varFrom* columns is expected.')
# da.playerDays

blockAssignment, dayPaddingStats = whr.makePlayerDayCountBlocks(da.players.playerDayCount, maxBlocks=5)
# if 'extDayCount' in da.players: del da.players['extDayCount']
da.players = da.players.join(blockAssignment.extDayCount, on='playerDayCount', validate='m:1', how='left')

assert (da.players.playerDayCount <= da.players.extDayCount).all()
pd.options.display.min_rows = 20
pd.options.display.max_rows = 60
# HTML(dayPaddingStats.to_html())

using cached data from cache/games.pickle
using cached data from cache/playerNames.pickle
using cached data from cache/playerDays.pickle


In [4]:
extPlayerDays = whr.extendPlayerDaysWithPaddingDays(da.players, da.playerDays)
extPlayerDays['extIx'] = range(len(extPlayerDays))

# Contains positional indices into `extendedPlayerDays`
da.playerDays['extIx'] = extPlayerDays.extIx.droplevel('extDayCount')

assert (extPlayerDays.iloc[da.playerDays.extIx].index.droplevel('extDayCount') == da.playerDays.index).all()
assert (extPlayerDays.isRealDay.iloc[da.playerDays.extIx].all())

reordPlayerDays = extPlayerDays[extPlayerDays.isRealDay].copy()
reordPlayerDays['reordIx'] = range(len(reordPlayerDays))
da.playerDays['reordIx'] = reordPlayerDays.reordIx.droplevel('extDayCount')
extPlayerDays['reordIx'] = reordPlayerDays.reordIx

assert (reordPlayerDays.iloc[da.playerDays.reordIx].index.droplevel('extDayCount') == da.playerDays.index).all()
assert len(reordPlayerDays) == len(da.playerDays)
assert (extPlayerDays['reordIx'].isna() ^ extPlayerDays.isRealDay).all()

    # extPlayerDays : DataFrame
    # """ 
    # Index:
    #     extDayCount : int
    #     player : int
    #     day : datetime64
    # Columns:
    #     sdev: float, Standard deviation of natural rating change compared to previous day (prior)
    #     isRealDay: bool, True iff it is not a padding day
    # """

    # reordPlayerDays : DataFrame
    # """ 
    #     Ordered like `extPlayerDays`, but restricted to real days
    # Index:
    #     extDayCount : int
    #     player : int
    #     day : datetime64
    # Columns:
    #     sdev: float, Standard deviation of natural rating change compared to previous day (prior)
    # """


In [5]:
if False:
    for ig, g in enumerate(da.games.index):
        day = da.games.day[g]

        winners = da.games.winners[g]
        losers = da.games.losers[g]
        winnerRatingIxs = da.playerDays.extIx.loc[(winners,day)]
        loserRatingIxs  = da.playerDays.extIx.loc[(losers,day)]

        assert (extPlayerDays.index.get_level_values('player')[winnerRatingIxs] == winners).all()
        assert (extPlayerDays.index.get_level_values('player')[loserRatingIxs] == losers).all()
        assert (extPlayerDays.index.get_level_values('day')[winnerRatingIxs] == day).all()
        assert (extPlayerDays.index.get_level_values('day')[loserRatingIxs] == day).all()

In [6]:
import pymc as pm
import pytensor.tensor as pt
import pytensor as pt0
import pytensor.sparse as pts
import pytensor.d3viz as d3v
import math

whr.setup_pytensor()

separateVirtualGames = False


In [7]:
def assertSlice(x : Series) -> slice:
    s = slice(x.iat[0], x.iat[-1] + 1)
    assert (x==range(s.start, s.stop)).all()
    return s

In [8]:
# %%pyinstrument

coords = {
        "player": da.players.name, 
        # "playerDay": playerDays.index.map(lambda t: "%s, %s" % (players.name.at[t[1]], t[2])),
        "reordPlayerDay": reordPlayerDays.index.map(lambda t: "%s, %s" % (da.players.name.at[t[1]], t[2])),
        "extPlayerDay": extPlayerDays.index.map(lambda t: "%s, %s" % (da.players.name.at[t[1]], t[2])),
        "game": da.games.index
}

gameCountReal = len(da.games)
gameCountEff = gameCountReal  + (0 if separateVirtualGames else 2 * len(da.players))

pdc = len(da.playerDays)
epdc = len(extPlayerDays)
hasPadding = pdc != epdc

basic_model = pm.Model(coords=coords, check_bounds=False)

with basic_model:
    # rw = pm.GaussianRandomWalk('RW', sigma = 1, steps = 4, init_dist = pm.Normal.dist(0, 0.001))
    playerDayRatingSDevs = pm.ConstantData('playerDayRatingSDevs ', reordPlayerDays.sdev , dims=("reordPlayerDay",))

    # def ratingsScanFunc(isNext, incr, prev):
    #     return pt.choose(isNext, [prev + incr, pt.constant(0, dtype='floatX')])

    increments = pm.Normal('increments', mu=0, sigma = playerDayRatingSDevs, dims="reordPlayerDay")

    if hasPadding:
        extIncrementsReal = pt.zeros(epdc)[reordPlayerDays.extIx]
        extIncrementsReal.name = 'extIncrements[real_days]'

        extIncrements = pt.inc_subtensor(extIncrementsReal, increments, set_instead_of_inc=True, ignore_duplicates=True)
        extIncrements.name = 'extIncrements'
    else:
        extIncrements = increments

    ratingsBuilder = pt.zeros(pdc)
    ratingsBuilder.name = 'ratingsBuilder_empty'

    # A player 'block' is a subset of players each having the same number of (extended) days played.
    # ("extended" = Some players had dummy days added in order to have fewer blocks and thus better vectorization)
    # We can use just a single `cumsum` for each block.
    for extDayCount in extPlayerDays.index.unique('extDayCount'):
        ext = extPlayerDays.loc[extDayCount]
        reord = reordPlayerDays.loc[extDayCount]

        blockPlayers = ext.index.unique('player')
        playerCount = len(blockPlayers)

        assert len(ext) == extDayCount * playerCount

        reordSlice = assertSlice(reord.reordIx)
        extSlice = assertSlice(ext.extIx)

        if(extDayCount > 1):
            incrs = extIncrements[extSlice].reshape((playerCount, extDayCount))
            ratingsBlock = pt.cumsum(incrs, axis=1).reshape((len(ext),))
            ratingsBlockReal = ratingsBlock[reord.extIx - reord.extIx.iat[0]]
            ratingsBlock.name = f'ratingsBlock(edc={extDayCount})'
        else:
            assert (reord.reordIx == ext.extIx).all()
            ratingsBlock = increments[reordSlice]
            ratingsBlockReal = ratingsBlock

        ratingsBlockReal.name = f'ratingsBlock[real_days](edc={extDayCount})'

        ratingsBuilder = pt.inc_subtensor(ratingsBuilder[reordSlice],
                                          ratingsBlockReal, 
                                          set_instead_of_inc=True, ignore_duplicates=True)

        ratingsBuilder.name = f'ratingsBuilder(edc={extDayCount})'

    ratings = pm.Deterministic('ratings', ratingsBuilder, dims="reordPlayerDay")
    # ratings = pm.Deterministic('ratings', extRatings[realExtIxsSorted] if hasPadding else extRatings, dims="reordPlayerDay")
    # ratings = extRatings[realExtIxsSorted] if hasPadding else extRatings

    ratingsToGameLogitsMatrix, firstDayIndices = whr.createRatingsToGameLogitsMatrix(
        da,
        ratingIxLookup=da.playerDays.reordIx,
        ratingCount=pdc,
        dtype = pt0.config.floatX,
        selfCheck = True, separateVirtualGames=separateVirtualGames)

    ratingsToGameLogits = pts.constant(ratingsToGameLogitsMatrix, name='ratingsToGameLogits')

    gameLogits = pts.structured_dot(
        ratingsToGameLogits, 
        ratings.dimshuffle(0,'x')
    ).dimshuffle(0)

    useOutcomesPotential = True
    whr.makeOutcomes(da, ratings, gameLogits, firstDayIndices, usePotential=useOutcomesPotential)

basic_model


In [9]:
# (playerDayRatingSDevs.dtype, innovations.dtype, ratings.dtype, ratingsToGameLogits.dtype, gameLogits.dtype, outcomes.dtype)
# virtWins.dtype


In [10]:
# For perf stat logging
modelDescription = {
    'games' : len(da.games),
    'realDays' : len(da.playerDays),
    'paddingDays' : epdc - len(da.playerDays),
    'players' : len(da.players),
    'cumsumBlocks' : len(extPlayerDays.index.unique('extDayCount')),
    'virtualGameOutcomeCount' : 2 * len(da.players),
    'ratingsToGameLogitsMatrix.count_nonzero' : ratingsToGameLogitsMatrix.count_nonzero(),
    'ratingsToGameLogitsMatrix.type' : repr(ratingsToGameLogits),
    'floatX' : pt0.config.floatX,
    'separateVirtualGames ' : separateVirtualGames,
    'ptconfig.openmp' : pt0.config.openmp,
    'padIncrements' : False,
    'padRatings' : False,
    'useOutcomesPotential' : useOutcomesPotential,
    'comment' : "eliminate extRatings"
}

# modelDescription


In [11]:
# basic_model.debug(fn='logp')
# basic_model.debug(fn='dlogp')
pt0.config.exception_verbosity='low'
# pt0.dprint(basic_model.compile_dlogp().f)

In [12]:
idata = whr.sample(basic_model, modelDescription, da, da.playerDays.reordIx)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [increments]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 169 seconds.


Sampling completed; extending perf stats


In [13]:
from IPython.display import HTML

perfStats = whr.loadPerfStats()
perfStats = pd.concat([DataFrame({'msPerSample': 1000 * perfStats.sampling_time / (perfStats.chains * perfStats.draws)} ) , perfStats], axis=1)
perfStats = perfStats[perfStats.draws==1000]
HTML(perfStats[-20:].to_html())

,msPerSample,created_at,sampling_time,chains,draws,games,realDays,paddingDays,players,cumsumBlocks,virtualGameOutcomeCount,ratingsToGameLogitsMatrix.count_nonzero,ratingsToGameLogitsMatrix.type,OMP_NUM_THREADS,p505371_start_mean,p505371_start_sdev,p505371_end_mean,p505371_end_sdev,first_date,arviz_version,inference_library,inference_library_version,tuning_steps,floatX,innovsToGameLogitsMatrix.count_nonzero,innovsToGameLogitsMatrix.type,lowerCumsumMatrix.count_nonzero,lowerCumsumMatrix.type,preMultiplyCumsumMatrix,separateVirtualGames,ptconfig.openmp,padIncrements,padRatings,discard_increments,isCustomDiffModel,usesDiffMatrix,useOutcomesPotential,nuts_sampler,installed_libs,ptconfig.blas__ldflags,ptconfig.lib__amblibm,init,comment
74,47.717831,2023-06-02T00:13:19.219767,190.871325,4,1000,7267,13114,0,2893,58,5786,52227,"SparseConstant{csr,float32,shape=(13053, 13114),nnz=52227}",2,1676.086182,181.514908,1682.205322,226.362595,2023-02-28 23:41:27,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,<NA>,<NA>,<NA>,False,True,False,False,<NA>,NaN,<NA>,True,pymc,libatlas-base-dev,-lblas,False,auto,Use inc_subtensor instead of join for cumsum blocks
75,28.323757,2023-06-02T00:17:43.537838,113.295028,4,1000,7267,13114,1838,2893,10,5786,52227,"SparseConstant{csr,float32,shape=(13053, 13114),nnz=52227}",2,1678.311523,183.847229,1683.945679,224.365082,2023-02-28 23:41:27,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,<NA>,<NA>,<NA>,False,True,False,False,<NA>,NaN,<NA>,True,pymc,libatlas-base-dev,-lblas,False,auto,Use inc_subtensor instead of join for cumsum blocks
76,28.239481,2023-06-02T02:22:08.277197,112.957922,4,1000,7267,13114,1838,2893,10,5786,52227,"SparseConstant{csr,float32,shape=(13053, 13114),nnz=52227}",2,1674.81604,175.220062,1686.449219,227.599899,2023-02-28 23:41:27,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,<NA>,<NA>,<NA>,False,True,False,False,<NA>,NaN,<NA>,True,pymc,libatlas-base-dev,-lblas,False,auto,Use inc_subtensor instead of join for cumsum blocks
77,126.159937,2023-06-03T01:11:57.955206,504.639747,4,1000,14216,24438,0,4548,87,9096,92980,"SparseConstant{csr,float32,shape=(23312, 24438),nnz=92980}",2,1640.843506,179.977875,1799.115601,221.63179,2023-01-01 01:31:45,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,<NA>,<NA>,<NA>,False,True,False,False,<NA>,NaN,<NA>,True,pymc,libatlas-base-dev,-lblas,False,auto,Use inc_subtensor instead of join for cumsum blocks
78,82.30687,2023-06-03T01:41:36.465589,329.227481,4,1000,14216,24438,1430,4548,20,9096,92980,"SparseConstant{csr,float32,shape=(23312, 24438),nnz=92980}",2,1640.616333,183.211655,1799.908325,216.567642,2023-01-01 01:31:45,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,<NA>,<NA>,<NA>,False,True,False,False,<NA>,NaN,<NA>,True,pymc,libatlas-base-dev,-lblas,False,auto,Use inc_subtensor instead of join for cumsum blocks
79,72.853244,2023-06-03T01:49:31.158757,291.412974,4,1000,14216,24438,5332,4548,10,9096,92980,"SparseConstant{csr,float32,shape=(23312, 24438),nnz=92980}",2,1640.362793,182.80835,1796.358887,218.123886,2023-01-01 01:31:45,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,<NA>,<NA>,<NA>,False,True,False,False,<NA>,NaN,<NA>,True,pymc,libatlas-base-dev,-lblas,False,auto,Use inc_subtensor instead of join for cumsum blocks
80,72.701581,2023-06-03T02:07:33.288595,290.806322,4,1000,14216,24438,16849,4548,5,9096,92980,"SparseConstant{csr,float32,shape=(23312, 24438),nnz=92980}",2,1641.883789,182.049362,1795.574097,215.283234,2023-01-01 01:31:45,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,<NA>,<NA>,<NA>,False,True,False,False,<NA>,NaN,<NA>,True,pymc,libatlas-base-dev,-lblas,False,auto,Use inc_subtensor instead of join for cumsum blocks
81,66.401635,2023-06-03T02:18:57.471710,265.606541,4,1000,14216,24438,0,4548,<NA>,9096,92980,"SparseConstant{csr,float32,shape=(23312, 24438),nnz=92980}",2,1641.763306,190.806213,1797.758301,222.56427,2023-01-01 01:31:45,0.15.1,pymc,5.3.1,1000,float32,<NA>,<NA>,334640,"SparseConstant{csr,int8,shape=(24438, 24438),nnz=334640}",False,False,True,<NA>,<NA>,<NA>,NaN,<NA>,True,

^ PerfStats

In [ ]:
# pt0.config.mode = 'FAST_RUN'
# pt0.config.mode = 'DebugMode'
# pt0.config.profiling__time_thunks = True
# pt0.config.check_input = True # C compilation fails if False
pt0.config.profile_memory = False
# pt0.config.n_apply = 20
# pt0.config.profiling__min_memory_size = 10
profileStats = basic_model.profile([basic_model.logp().sum(), basic_model.dlogp()])
profileStats.summary(n_apply_to_print=20)

In [ ]:
profileStats = basic_model.profile([basic_model.dlogp()])
profileStats.summary()

In [ ]:
smmry = az.summary(idata, round_to=2)

In [ ]:
pt0.dprint(basic_model.compile_logp().f, print_view_map=True)

In [ ]:
# idata.to_netcdf('model-2023-04-var-0.1.netcdf')

In [ ]:
ratingIxLookup = da.playerDays.reordIx

import IPython
postRatings : DataFrame = idata.posterior['ratings'].mean(dim=["chain", "draw"]).to_dataframe()
medians = idata.posterior['ratings'].median(dim=["chain", "draw"])

postRatings.index = ratingIxLookup.sort_values().index
postRatings = postRatings.join(da.players.name, how='left', on='player').droplevel('player').set_index('name', append=True)

postRatings = postRatings.assign(mean_elo = whr.naturalRatingToElo(postRatings.ratings),
                                 median_elo = whr.naturalRatingToElo(medians))

eloOut = postRatings.mean_elo.groupby('name').agg(['mean','median','first','max','min','last']).round(0)
eloOut['range'] = eloOut['max'] - eloOut['min']
# IPython.display.HTML(eloOut.to_html())
# regulars = da.players[da.players.playerDayCount > 9]

# IPython.display.HTML(eloOut.sort_values('last',ascending=False)[:30].reset_index().to_html())

In [ ]:
import xarray as xa
incCorr = idata.posterior['increments'][:,:,-1000:].to_dataframe().unstack('reordPlayerDay').droplevel(0, axis=1).corr()
incCorr = incCorr.unstack()
incCorr = incCorr[incCorr.index.get_level_values(0) < incCorr.index.get_level_values(1)]
# largeCorr = incCorr[incCorr.abs() > 0.1]

In [ ]:
# incCorr.sort_values()

In [ ]:
import matplotlib as mpl
import matplotlib.axes as mpla
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

axs = az.plot_forest(idata, var_names = '[Rr]ating', filter_vars="regex", combined=True)

ax : mpla.Axes =  axs[0]
ax.minorticks_on()
ax.xaxis.set_major_locator(plticker.MultipleLocator(base=5.0))
ax.xaxis.set_minor_locator(plticker.MultipleLocator(base=1.0))
ax.xaxis.set_tick_params(which='both', top=True, labeltop=True, bottom=True, labelbottom=True)
# ax.xaxis.set_tick_params(which='minor', grid_color='black', grid_linewidth=1, grid_alpha=1, grid_linestyle='-')
ax.xaxis.grid(True, which='major', color='black')
ax.xaxis.grid(True, which='minor')





# profileStats = basic_model.profile(basic_model.dlogp())
# profileStats.summary()
# fn = basic_model.compile_logp()
# pass
# az.plot_pair(idata, var_names = 'rating', filter_vars="like", kind='kde')

In [ ]:
pm.model_to_graphviz(basic_model)

In [ ]:
playerDays = da.playerDays

In [ ]:
az.waic(idata)

In [ ]:
from IPython.display import SVG
from pytensor.printing import pydotprint

# postrw = idata.posterior["RW"]
# idata
# d3v.d3viz(basic_model.compile_logp().f, '/tmp/d3v.html')

pydotprint(basic_model.compile_dlogp().f, format='svg')

# SVG(pt0.printing.pydotprint(basic_model.logp(), return_image=True, format='svg'))

# pt0.dprint(basic_model.compile_logp().f, print_storage=True, print_view_map=False);

#az.plot_forest(idata)
# print(pt0.pp(basic_model.logp()))

In [ ]:
import xarray as xa
# az.plot_trace(idata, combined=True); #, coords={"RW_dim_0":range(1,5)});
postRatings : xa.DataArray = idata.posterior['ratings']
postRatings

In [ ]:
idata.posterior["alpha"].sel(draw=slice(0, 4))